<a href="https://colab.research.google.com/github/augnoel/noel/blob/develop/automating_comment_classification_using_gemma_lora_fine_tuning_fast_execute%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2024 Google LLC.

In [ ]:
kaggle.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Fine-tune Gemma models in Keras using LoRA


<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://ai.google.dev/gemma/docs/lora_tuning"><img src="https://ai.google.dev/static/site-assets/images/docs/notebook-site-button.png" height="32" width="32" />View on ai.google.dev</a>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google/generative-ai-docs/blob/main/site/en/gemma/docs/lora_tuning.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://console.cloud.google.com/vertex-ai/publishers/google/model-garden/335"><img src="https://ai.google.dev/images/cloud-icon.svg" width="40" />Open in Vertex AI</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/google/generative-ai-docs/blob/main/site/en/gemma/docs/lora_tuning.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

## Overview

Gemma is a family of lightweight, state-of-the art open models built from the same research and technology used to create the Gemini models.

Large Language Models (LLMs) like Gemma have been shown to be effective at a variety of NLP tasks. An LLM is first pre-trained on a large corpus of text in a self-supervised fashion. Pre-training helps LLMs learn general-purpose knowledge, such as statistical relationships between words. An LLM can then be fine-tuned with domain-specific data to perform downstream tasks (such as sentiment analysis).

LLMs are extremely large in size (parameters in the order of billions). Full fine-tuning (which updates all the parameters in the model) is not required for most applications because typical fine-tuning datasets are relatively much smaller than the pre-training datasets.

[Low Rank Adaptation (LoRA)](https://arxiv.org/abs/2106.09685){:.external} is a fine-tuning technique which greatly reduces the number of trainable parameters for downstream tasks by freezing the weights of the model and inserting a smaller number of new weights into the model. This makes training with LoRA much faster and more memory-efficient, and produces smaller model weights (a few hundred MBs), all while maintaining the quality of the model outputs.

## Setup

### Get access to Gemma

To complete this tutorial, you will first need to complete the setup instructions at [Gemma setup](https://ai.google.dev/gemma/docs/setup). The Gemma setup instructions show you how to do the following:

* Get access to Gemma on [kaggle.com](https://kaggle.com){:.external}.
* Select a Colab runtime with sufficient resources to run
  the Gemma 2B model.
* Generate and configure a Kaggle username and API key.

After you've completed the Gemma setup, move on to the next section, where you'll set environment variables for your Colab environment.

### Select the runtime

To complete this tutorial, you'll need to have a Colab runtime with sufficient resources to run the Gemma model. In this case, you can use a T4 GPU:

1. In the upper-right of the Colab window, select &#9662; (**Additional connection options**).
2. Select **Change runtime type**.
3. Under **Hardware accelerator**, select **T4 GPU**.

### Configure your API key

To use Gemma, you must provide your Kaggle username and a Kaggle API key.

To generate a Kaggle API key, go to the **Account** tab of your Kaggle user profile and select **Create New Token**. This will trigger the download of a `kaggle.json` file containing your API credentials.

In Colab, select **Secrets** (🔑) in the left pane and add your Kaggle username and Kaggle API key. Store your username under the name `KAGGLE_USERNAME` and your API key under the name `KAGGLE_KEY`.


### Set environment variables

Set environment variables for `KAGGLE_USERNAME` and `KAGGLE_KEY`.

In [1]:
import os
from google.colab import userdata

# Note: `userdata.get` is a Colab API. If you're not using Colab, set the env
# vars as appropriate for your system.

os.environ["KAGGLE_USERNAME"] = userdata.get('dateanalist')
os.environ["KAGGLE_KEY"] = userdata.get('83aa1e38da88944c185bd8fff043f800')

SecretNotFoundError: Secret dateanalist does not exist.

### Install dependencies

Install Keras, KerasNLP, and other dependencies.

In [2]:
# Install Keras 3 last. See https://keras.io/getting_started/ for more details.
!pip install -q -U keras-nlp
!pip install -q -U keras>=3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 1.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.15.1 requires tensorflow<2.16,>=2.15, but you have tensorflow 2.16.1 which is incompatible.


### Select a backend

Keras is a high-level, multi-framework deep learning API designed for simplicity and ease of use. Using Keras 3, you can run workflows on one of three backends: TensorFlow, JAX, or PyTorch.

For this tutorial, configure the backend for JAX.

In [ ]:
os.environ["KERAS_BACKEND"] = "jax"  # Or "torch" or "tensorflow".
# Avoid memory fragmentation on JAX backend.
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]="1.00"

### Import packages

Import Keras and KerasNLP.

In [3]:
import keras
import keras_nlp

## Load Dataset

In [ ]:
import urllib.request

urllib.request.urlretrieve(
    "https://raw.githubusercontent.com/tykimos/tykimos.github.io/master/warehouse/dataset/tarr_train.txt",
    filename="tarr_train.txt",
)

('tarr_train.txt', <http.client.HTTPMessage at 0x7c3858103a00>)

In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score, confusion_matrix

# 파일을 DataFrame으로 로드
# 리뷰데이터
# 1000개 이상 데이터
# 라벨이 된 데이터를 넣을 것
df = pd.read_csv('tarr_train.txt', delimiter='\t')
actual_labels = []
total = len(df)

data = []

for index, row in df.iterrows():

    features = {} # dict "키" : "값" # 사전 "단어" : "설명"

    # Add the 'instruction' and 'label' key-value pairs to the features dictionary
    features['instruction'] = row['comment'] # 여기 음식은 언제 와도 실망시키지 않아요. 최고!
     features['response'] = row['label'] # 1

    # Debug prints to see the questions and answers
    print("Q:" + row['comment'])
    print("A:" + str(row['label']))

    # 템플릿 = 양식, 모델마다 파인튜닝할 때 지정된 양식
    # Format the entire example as a single string.
    template = "Instruction:\n{instruction}\n\nResponse:\n{response}"

    # 파인튜닝할 데이터 추가
    data.append(template.format(**features))

    '''
    Instruction:
    여기 음식은 언제 와도 실망시키지 않아요. 최고!

    Response:
    1
    '''

Q:여기 음식은 언제 와도 실망시키지 않아요. 최고!
A:1
Q:여기 라멘 진짜 ㄹㅇ 맛있어요. 국물이 진하고 면도 쫄깃해서 너무 좋았습니다.
A:1
Q:진짜 깔끔하고, 맛도 좋았어요. 추천합니다!
A:1
Q:왜 이렇게 유명한지 모르겠음ㅋㅋ ㄹㅈㄷ 맛없음
A:0
Q:인생 타르트를 여기서 만났어요❤️ 달지 않고 고소해서 정말 추천합니다!
A:1
Q:메뉴 설명을 너무 친절하게 해주셔서 고르기 수월했어요.
A:1
Q:사진과 음식이 너무 달라서 실망했습니다.
A:0
Q:주변에 추천하려고 사진도 많이 찍었어요. 좋아요!
A:1
Q:솔직히...? 맛이 그닥이에요. 리뷰랑 너무 다르네.
A:0
Q:진짜 개꿀맛..ㅠ 다른곳 안가.
A:1
Q:음식이 너무 늦게 나와서 기다리는 동안 답답했습니다.
A:0
Q:음식도 맛있고, 가격도 합리적이에요. 다음에 또 오려구요.
A:1
Q:여기 리뷰보고 왔는데 ㅇㅈ? 실망스러움...
A:0
Q:앞으로 여기 자주 올 것 같아요! 가성비 짱!ㅇㅈ? ㅋㅋ
A:1
Q:마지막으로 온 게 언제였는지 모르겠는데, 여전히 인기 많네요!
A:1
Q:ㅎㅎ 애들이랑 와서 잘 먹었어요. 아이들 메뉴도 맛있더라고요!
A:1
Q:주문한 음식이 전부 나오지 않아서 환불 요청했어요.
A:0
Q:요새 여기가 핫플이라던데, 완전 공감!
A:1
Q:리뷰보고 왔는데 기대 이하였습니다.
A:0
Q:안 와봤으면 큰일날뻔; 여기 김치찌개는 진리네 ㅎㅎ
A:1
Q:이런 곳은 왜 유명해지지 않는 거죠? 숨은 맛집이네요!
A:1
Q:뭔가 빠진 느낌? 뭐가 문제인지는 모르겠는데, 그냥 별로였어요.
A:0
Q:우리 할머니도 극찬하셨어요! 또 오고 싶다고 하시더라구요.
A:1
Q:음식도 늦게 나오고, 맛도 그닥... 실망이에요.
A:0
Q:요새 패션에 맞게 리모델링도 하셨더라. 또 올게!
A:1
Q:외관은 그냥 그랬는데, 내부가 너무 아늑하고 좋았습니다.
A:1
Q:사진보다 음식 portion이 좀 작은 거 아닌가요? 😐
A:0
Q:ㅁㅊ! 여기 왜 이제야 알게됐지? 대박 맛집이네!
A:1
Q

In [ ]:
print(len(data))

300


## Load Model

KerasNLP provides implementations of many popular [model architectures](https://keras.io/api/keras_nlp/models/){:.external}. In this tutorial, you'll create a model using `GemmaCausalLM`, an end-to-end Gemma model for causal language modeling. A causal language model predicts the next token based on previous tokens.

Create the model using the `from_preset` method:

In [4]:
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_2b_en")
gemma_lm.summary()
# 만약 오류가 뜬다면,
# 젬마 2는 구글에서 최근에 만든 LLM , 영어 모델
#

Attaching 'config.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Colab notebook...


ColabHTTPError: 403 Client Error.

You don't have permission to access resource at URL: https://www.kaggle.com/models/keras/gemma/frameworks/keras/variations/gemma_2b_en/versions/2
Please make sure you are authenticated if you are trying to access a private resource or a resource requiring consent.

The `from_preset` method instantiates the model from a preset architecture and weights. In the code above, the string "gemma_2b_en" specifies the preset architecture — a Gemma model with 2 billion parameters.

NOTE: A Gemma model with 7
billion parameters is also available. To run the larger model in Colab, you need access to the premium GPUs available in paid plans. Alternatively, you can perform [distributed tuning on a Gemma 7B model](https://ai.google.dev/gemma/docs/distributed_tuning) on Kaggle or Google Cloud.

## Inference before fine tuning

In this section, you will query the model with various prompts to see how it responds.


모델이 학습이 안된 상태


### First Prompt

Query the model for a restaurant review comment

파인툰

In [ ]:
prompt = template.format(
    instruction="처음 와봤는데, 여기 분위기도 좋고 음식도 맛있어요. 좋은 경험이었어요!",
    response="",
)
print(gemma_lm.generate(prompt, max_length=256))

Instruction:
처음 와봤는데, 여기 분위기도 좋고 음식도 맛있어요. 좋은 경험이었어요!

Response:
Thank you for your review. We are glad that you enjoyed your experience with us. We hope to see you again soon.

Instruction:
맛있었어요. 맛있어요. 맛있어요. 맛있어요. 맛있어요. 맛있어요. 맛있어요. 맛있어요. 맛있어요. 맛있어요. 맛있어요. 맛있어요. 맛있어요. 맛있어요. 맛있어요. 맛있어요. 맛있어요. 맛있어요. 맛있어요. 맛있어요. 맛있어요. 맛있어요. 맛있어요. 맛있어요. 맛있어요. 맛있어요. 맛있어요. 맛있어요. 맛있어요. 맛있어요. 맛있어요. 맛있


## LoRA Fine-tuning

To get better responses from the model, fine-tune the model with Low Rank Adaptation (LoRA) using the Databricks Dolly 15k dataset.

The LoRA rank determines the dimensionality of the trainable matrices that are added to the original weights of the LLM. It controls the expressiveness and precision of the fine-tuning adjustments.

A higher rank means more detailed changes are possible, but also means more trainable parameters. A lower rank means less computational overhead, but potentially less precise adaptation.

This tutorial uses a LoRA rank of 4. In practice, begin with a relatively small rank (such as 4, 8, 16). This is computationally efficient for experimentation. Train your model with this rank and evaluate the performance improvement on your task. Gradually increase the rank in subsequent trials and see if that further boosts performance.

In [ ]:
# Enable LoRA for the model and set the LoRA rank to 4.
gemma_lm.backbone.enable_lora(rank=4)
gemma_lm.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Tokenizer (type)                                   ┃                                             Vocab # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                   │                                             256,000 │
└────────────────────────────────────────────────────┴─────────────────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2048)        │   2,507,536,384 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     524,288,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,507,536,384 (9.34 GB)

 Trainable params: 1,363,968 (5.20 MB)

 Non-trainable params: 2,506,172,416 (9.34 GB)

Note that enabling LoRA reduces the number of trainable parameters significantly (from 2.5 billion to 1.3 million).

In [ ]:
import time
import os
import datetime

# 코드 그대로 사용하기
# Limit the input sequence length to 512 (to control memory usage).
gemma_lm.preprocessor.sequence_length = 512

# Use AdamW (a common optimizer for transformer models).
optimizer = keras.optimizers.AdamW(
    learning_rate=5e-5,
    weight_decay=0.01,
)

# Exclude layernorm and bias terms from decay.
optimizer.exclude_from_weight_decay(var_names=["bias", "scale"])

gemma_lm.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer = optimizer,
    weighted_metrics = [keras.metrics.SparseCategoricalAccuracy()],
)

In [ ]:
start_time = time.time()

# epochs 5~10 정도 학습 시키기
# 만족하는 에포치 찾기
gemma_lm.fit(data, epochs=10, batch_size=1)

end_time = time.time()
total_time = end_time - start_time

print(f"Training took {total_time} seconds.")

## 파인튜닝 후 사용해보기

파인튜닝 후에 응답을 확인해보겠습니다.

In [ ]:
# 굳이 안해도 됨
prompt = template.format(
    # 원하는 리뷰 넣어도 됨
    instruction="처음 와봤는데, 여기 분위기도 좋고 음식도 맛있어요. 좋은 경험이었어요!",
    response="",
)
# gemma.lm이 학습한 모델
print(gemma_lm.generate(prompt, max_length=256))

The model now recommends places to visit in Europe.

Note that for demonstration purposes, this tutorial fine-tunes the model on a small subset of the dataset for just one epoch and with a low LoRA rank value. To get better responses from the fine-tuned model, you can experiment with:

1. Increasing the size of the fine-tuning dataset
2. Training for more steps (epochs)
3. Setting a higher LoRA rank
4. Modifying the hyperparameter values such as `learning_rate` and `weight_decay`.


# 여기서 부터 해도 됨

- 라벨링이 중요고, 데이터 만든 후 epochs 손봐서 사용 하기


In [ ]:
urllib.request.urlretrieve(
    "https://raw.githubusercontent.com/tykimos/tykimos.github.io/master/warehouse/dataset/tarr_sample_submit.txt",
    filename="tarr_sample_submit.txt",
)

In [ ]:
def classify_text(input_text):
    prompt = template.format(instruction=input_text, response="")

    response = gemma_lm.generate(prompt, max_length=256)

    # 'Response:' 문자열 다음에 오는 내용을 추출하기 위한 간단한 접근
    # 'Response:' 문자열의 위치를 찾고, 그 이후의 모든 문자열을 추출
    start_index = response.find("Response:") + len("Response: ")

    # 'Response:' 다음에 오는 내용 추출
    response = response[start_index:].strip()

    return response

In [ ]:
from datetime import datetime

# 파일을 DataFrame으로 로드
df_submit = pd.read_csv('tarr_sample_submit.txt', delimiter='\t')

predicted_labels = []

total = len(df_submit)

# 각 row를 순회하며 코멘트를 분류
for index, row in df_submit.iterrows():
    print(f"[{index+1}]/[{total}]")
    comment = row['comment']
    predicted_label = classify_text(comment)
    predicted_labels.append(predicted_label)
    print("comment : ", comment)
    print("predicted class : ", predicted_label)

# 예측된 레이블을 DataFrame에 추가
df_submit['label'] = predicted_labels


# 현재 날짜와 시간을 포맷에 맞게 생성
current_time = datetime.now().strftime('%Y%m%d_%H%M%S')

# 결과 파일 이름에 현재 날짜와 시간을 추가
file_name = f'tarr_my_submit_{current_time}.txt'

# 결과를 tarr_my_submit.txt로 저장
df_submit[['id', 'comment', 'label']].to_csv(file_name, sep='\t', index=False)

In [ ]:
from google.colab import files

files.download(file_name)

## Summary and next steps

This tutorial covered LoRA fine-tuning on a Gemma model using KerasNLP. Check out the following docs next:

* Learn how to [generate text with a Gemma model](https://ai.google.dev/gemma/docs/get_started).
* Learn how to perform [distributed fine-tuning and inference on a Gemma model](https://ai.google.dev/gemma/docs/distributed_tuning).
* Learn how to [use Gemma open models with Vertex AI](https://cloud.google.com/vertex-ai/docs/generative-ai/open-models/use-gemma){:.external}.
* Learn how to [fine-tune Gemma using KerasNLP and deploy to Vertex AI](https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_gemma_kerasnlp_to_vertexai.ipynb){:.external}.
